In [1]:
import arcpy
import os

# Prepare Static Features
In addition to weather, and potential other dynamic data feeds, this notebook computes the static features. These include information about the roads that doesn't change very often. This is mostly things like the shape of the road, the population density around the road, locations of intersections, etc. This notebook heavily uses Arcpy to perform this processing. If you prefer, you can do this exact same analysis in ArcGIS Pro, using this notebook as a guide through the data and geoprocessing tools. You can also visualize these results in ArcGIS Pro by loading the `utah.gdb` into your project and exploring the data

<span class="mark">This notebook will take around an hour to run.</span> 

<span class="mark">**NOTE: ArcGIS Pro Advanced License Required for extensively used 'Near' tool.**</span>  
<span class="mark">**NOTE: Utah crashes_2010-2017.csv needs to be unzipped to run this notebook.
    **</span>

# Create GDB
All data is remote, but some will be processed and saved to a file GDB. Create this GDB is it doesn't exist and set this as the workspace

In [2]:
if not os.path.exists('utah.gdb'):
    arcpy.management.CreateFileGDB('.','utah.gdb')

# workspace
arcpy.env.workspace = r'./utah.gdb'
arcpy.env.overwriteOutput = True

# Define data URLs/Paths
The below cell contains all the data needed. All data is remote except **collisions_csv** and **weather_stations_csv**. The weather stations csv was created by `1_get_weather.ipynb`

In [3]:
# AADT, Intersections, Billboards
aadt_url = 'https://services1.arcgis.com/MdyCMZnX1raZ7TS3/arcgis/rest/services/Utah_AADT_2013/FeatureServer/0'
intersections_url = 'https://maps.udot.utah.gov/arcgis/rest/services/FI_Mandli2012/MapServer/5'
billboards_url = 'https://maps.udot.utah.gov/arcgis/rest/services/FI_Mandli2012/MapServer/2'
signals_url = 'https://maps.udot.utah.gov/arcgis/rest/services/FI_Mandli2012/MapServer/14'

population_url = 'http://services2.arcgis.com/gyfpgFh2Wj2gglYD/arcgis/rest/services/Census2010/FeatureServer/1'

collisions_csv = r'Utah crashes_2010-2017.csv'
weather_stations_csv = 'utah_stations.csv'

# Roads data
major_roads = 'http://services1.arcgis.com/99lidPhWCzftIe9K/arcgis/rest/services/Utah_Major_Road_Centerlines_(Statewide)/FeatureServer/0'
minor_roads = 'http://services1.arcgis.com/99lidPhWCzftIe9K/arcgis/rest/services/Utah_SGID_Road_Centerlines_(Statewide)/FeatureServer/0'

# Copy the remote major roads features into local GDB

In [4]:
# Copy the major road centerlines to the gdb
_ = arcpy.management.CopyFeatures(in_features=major_roads,out_feature_class='centerlines')

In [5]:
# Join AADT - simple process of joining centerline to the nearest segment. This has the possibility of being incorrect, but should be mostly accurate.
_ = arcpy.analysis.SpatialJoin('centerlines',
                           aadt_url,
                           'centerlines_aadt',
                           match_option='closest',
                           search_radius='200 feet')

In [6]:
# Delete identical roads. Sometimes the roads have the exact same polyline, keeping these will lower the accuracy of the model
# And there isn't really a distinction anyway.
_ = arcpy.management.DeleteIdentical('centerlines_aadt',xy_tolerance='1 feet',fields=['SHAPE'])

In [7]:
# Rename some fields for convenience
_ = arcpy.management.DeleteField('centerlines_aadt',['route_id','Shape_Leng','OBJECTID_12'])
_ = arcpy.management.AlterField('centerlines_aadt','aadt_vn','aadt',clear_field_alias=True)

# Merge major/minor roads

In [8]:
# Merge minor roads. We seperate these steps because we don't have AADT for minor roads.
_ = arcpy.management.Merge(['centerlines_aadt',minor_roads],'centerlines_merged')

# Rename/Delete Fields
For consistancy, readabiliy of the Python code, let's rename some of the fields and delete ones we don't care about

In [9]:
print([f.name for f in arcpy.ListFields('centerlines_merged')])

['OBJECTID_1', 'Shape', 'Join_Count', 'TARGET_FID', 'OBJECTID', 'ADDR_SYS', 'CARTOCODE', 'FULLNAME', 'L_F_ADD', 'L_T_ADD', 'R_F_ADD', 'R_T_ADD', 'PREDIR', 'STREETNAME', 'STREETTYPE', 'SUFDIR', 'ALIAS1', 'ALIAS1TYPE', 'ALIAS2', 'ALIAS2TYPE', 'ACSALIAS', 'ACSNAME', 'ACSSUF', 'ADDR_QUAD', 'USPS_PLACE', 'ZIPLEFT', 'ZIPRIGHT', 'COFIPS', 'ONEWAY', 'SPEED', 'VERTLEVEL', 'CLASS', 'HWYNAME', 'DOT_RTNAME', 'DOT_RTPART', 'DOT_F_MILE', 'DOT_T_MILE', 'MODIFYDATE', 'GLOBALID', 'COLLDATE', 'ACCURACY', 'SOURCE', 'NOTES', 'COUNIQUE', 'SURFTYPE', 'SURFWIDTH', 'DSTRBWIDTH', 'LOCALFUNC', 'MAINTJURIS', 'FED_RDID', 'STATUS', 'ACCESS', 'USAGENOTES', 'DOT_RTID', 'DOT_FUNC', 'DOT_COFUND', 'LOCALID', 'aadt', 'Shape_Length']


In [10]:
# Fields we don't need in the centerlines dataset for this
delete_fields = [
    'ADDR_SYS',
    'CARTOCODE',
    'FULLNAME',
    'L_F_ADD',
    'L_T_ADD',
    'R_F_ADD',
    'R_T_ADD',
    'STREETNAME',
    'ALIAS1',
    'ALIAS1TYPE',
    'ALIAS2',
    'ALIAS2TYPE',
    'ACSALIAS',
    'ACSNAME',
    'ACSSUF',
    'ADDR_QUAD',
    'USPS_PLACE', 
    'ZIPLEFT', 
    'ZIPRIGHT', 
    'COFIPS',
    'VERTLEVEL', 
    'CLASS', 
    'HWYNAME', 
    'DOT_RTNAME',
    'DOT_RTPART',
    'DOT_F_MILE', 
    'DOT_T_MILE', 
    'MODIFYDATE',
    'GLOBALID', 
    'COLLDATE', 
    'ACCURACY', 
    'SOURCE', 
    'NOTES', 
    'COUNIQUE', 
    'DSTRBWIDTH', 
    'LOCALFUNC',
    'MAINTJURIS', 
    'FED_RDID', 
    'STATUS', 
    'ACCESS', 
    'USAGENOTES',
    'DOT_RTID',
    'DOT_FUNC', 
    'DOT_COFUND',
    'LOCALID'
]
field_remap = {
    'PREDIR':'pre_dir',
    'STREETTYPE':'street_type',
    'SUFDIR':'suf_dir',
    'ONEWAY':'one_way',
    'SPEED':'speed_limit',
    'SURFTYPE':'surface_type',
    'SURFWIDTH':'surface_width'
}
_ = arcpy.management.DeleteField('centerlines_merged',delete_fields)
for field_name,remap_name in field_remap.items():
    arcpy.management.AlterField('centerlines_merged',field_name,remap_name,clear_field_alias=True)

In [11]:
print([f.name for f in arcpy.ListFields('centerlines_merged')])

['OBJECTID_1', 'Shape', 'Join_Count', 'TARGET_FID', 'OBJECTID', 'pre_dir', 'street_type', 'suf_dir', 'one_way', 'speed_limit', 'surface_type', 'surface_width', 'aadt', 'Shape_Length']


We will reference *segment_id* as the unique identifier for a road segment

In [12]:
# Add a segment_id field to use instead of OBJECTID_1
_ = arcpy.management.AddField('centerlines_merged','segment_id','Long')
_ = arcpy.management.CalculateField('centerlines_merged','segment_id','!OBJECTID_1!')

# Road Segment Spatial Features
There are several fields to add to the data to enrich. Some will be calculated off of the geometries, some off of proximity to features in other datasets

In [13]:
# Now we add some calculated fields:

fields = [
    ['sinuosity','Double'],
    ['euclidean_length','Double'],
    ['segment_length','Double'],
    ['at_intersection','Short'],
    ['near_billboard','Short'],
    ['road_orient_approx','Double'],
    ['proximity_to_signal','Double'],
    ['proximity_to_billboard','Double'],
    ['proximity_to_nearest_intersection','Double'],
    ['proximity_to_major_road','Double']
]

_ = arcpy.management.AddFields('centerlines_merged',fields)

In [14]:
# Calc Sinuosity
code_block = \
'''
import math
def getSinuosity(shp):
    x0 = shp.firstPoint.x
    y0 = shp.firstPoint.y
    x1 = shp.lastPoint.x
    y1 = shp.lastPoint.y

    euclid = math.sqrt((x0-x1)**2 + (y0-y1)**2)
    length = shp.length
    if euclid > 0:
        return length/euclid
    return 1.0
'''
_ = arcpy.management.CalculateField('centerlines_merged','sinuosity','getSinuosity(!Shape!)',code_block=code_block)

In [15]:
# Calc Euclidean Length
code_block = \
'''
import math
def getEuclideanLength(shp):
    x0 = shp.firstPoint.x
    y0 = shp.firstPoint.y
    x1 = shp.lastPoint.x
    y1 = shp.lastPoint.y

    euclid = math.sqrt((x0-x1)**2 + (y0-y1)**2)
    return euclid
'''
_ = arcpy.management.CalculateField('centerlines_merged','euclidean_length','getEuclideanLength(!Shape!)',code_block=code_block)

In [16]:
# Calc segment_length
_ = arcpy.management.CalculateField('centerlines_merged','segment_length','!Shape_Length!')

In [17]:
# Calc Road Orientation
code_block = \
'''
import math
def getOrientation(shp):
    x0 = shp.firstPoint.x
    y0 = shp.firstPoint.y
    x1 = shp.lastPoint.x
    y1 = shp.lastPoint.y

    angle = math.atan2(y1-y0,x1-x0)
    if angle < 0:
        angle += math.pi
    return angle
'''
_ = arcpy.management.CalculateField('centerlines_merged','road_orient_approx','getOrientation(!Shape!)',code_block=code_block)

In [18]:
# Calc at_intersection
_ = arcpy.analysis.Near('centerlines_merged',
                        intersections_url,
                        search_radius='150 feet')
_ = arcpy.management.CalculateField('centerlines_merged','at_intersection','!NEAR_DIST! >= 0')
_ = arcpy.management.DeleteField('centerlines_merged',['NEAR_DIST','NEAR_FID'])

In [19]:
# Calc proximity to intersection
_ = arcpy.analysis.Near('centerlines_merged',
                        intersections_url)
_ = arcpy.management.CalculateField('centerlines_merged','proximity_to_nearest_intersection','!NEAR_DIST!')
_ = arcpy.management.DeleteField('centerlines_merged',['NEAR_DIST','NEAR_FID'])

In [20]:
# Calc near_billboard
_ = arcpy.analysis.Near('centerlines_merged',
                        billboards_url,
                        search_radius='300 feet')
_ = arcpy.management.CalculateField('centerlines_merged','near_billboard','!NEAR_DIST! >= 0')
_ = arcpy.management.DeleteField('centerlines_merged',['NEAR_DIST','NEAR_FID'])

In [21]:
# Calc proximity to billboard
_ = arcpy.analysis.Near('centerlines_merged',
                        billboards_url)
_ = arcpy.management.CalculateField('centerlines_merged','proximity_to_billboard','!NEAR_DIST!')
_ = arcpy.management.DeleteField('centerlines_merged',['NEAR_DIST','NEAR_FID'])

In [22]:
# Calc proximity_to_signal
_ = arcpy.analysis.Near('centerlines_merged',
                        signals_url)
_ = arcpy.management.CalculateField('centerlines_merged','proximity_to_signal','!NEAR_DIST!')
_ = arcpy.management.DeleteField('centerlines_merged',['NEAR_DIST','NEAR_FID'])

In [23]:
# Calc proximity_to_major_road
_ = arcpy.analysis.Near('centerlines_merged',
                        'centerlines')
_ = arcpy.management.CalculateField('centerlines_merged','proximity_to_major_road','!NEAR_DIST!')
_ = arcpy.management.DeleteField('centerlines_merged',['NEAR_DIST','NEAR_FID'])

In [24]:
# Load collisions

# Projected Coords UTM NAD1983 Zone 12N
in_sr = arcpy.SpatialReference(26912)

_ = arcpy.management.MakeXYEventLayer(collisions_csv,
        out_layer='collisions',
        in_x_field='Longitude',
        in_y_field='Latitude',
        spatial_reference=in_sr)

# Projected Coords Web Mercator
out_sr = arcpy.Describe('centerlines_merged').spatialReference

_ = arcpy.management.Project('collisions','collisions_projected',out_sr)

# Population Density

In [25]:
_ = arcpy.management.CopyFeatures(in_features=population_url,out_feature_class='population')
_ = arcpy.management.AddFields('population',[['population_density','Double']])

In [26]:
print([f.name for f in arcpy.ListFields('population')])

['OBJECTID', 'Shape', 'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'BLOCKCE10', 'GEOID10', 'NAME10', 'FUNCSTAT10', 'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'LOGRECNO', 'POP100', 'HU100', 'P0020001', 'P0020002', 'P0020003', 'P0020004', 'P0020005', 'P0020006', 'P0020007', 'P0020008', 'P0020009', 'P0020010', 'MTFCC', 'P0010011', 'P0010012', 'P0010013', 'P0010014', 'MHI', 'TractDashBG', 'Shape_Length', 'Shape_Area', 'population_density']


In [27]:
code_block = \
'''
def getDensity(pop,area):
    try:
        return pop*10000.0/area
    except:
        return 0.0
'''
_ = arcpy.management.CalculateField('population','population_density','getDensity(!P0020001!,!Shape_Area!)',code_block=code_block)

In [28]:
field_mappings = arcpy.FieldMappings()
field_mappings.addTable('centerlines_merged')
features_mapping = arcpy.FieldMap()
features_mapping.addInputField('population','population_density')
field_mappings.addFieldMap(features_mapping)

_ = arcpy.analysis.SpatialJoin('centerlines_merged',
                               'population',
                               'centerlines_merged_with_pop',
                               join_type='KEEP_ALL',
                               match_option='intersects',
                               field_mapping=field_mappings)

# Assign Weather Stations to Segments
We create the Thiessen polygons for the weather station. A better way to do this is to interpolate the weather data to each road, but with thousdands of hours to perform this interpolation for, we fall back on this simple approach for the sake of this demo. Feel free to try out various interpolation techniques! Empirical Bayesian Kriging is a extremely powerful interpolation techniques available through the spatial statistics toolbox. A lighter weight, but less accurate option would be to do inverse distance interpolation at each road.

In [29]:
in_sr = arcpy.SpatialReference(4326)
_ = arcpy.management.MakeXYEventLayer(weather_stations_csv,
        out_layer='weather_station_eventlayer',
        in_x_field='LON',
        in_y_field='LAT',
        spatial_reference=in_sr)
res = arcpy.management.CopyFeatures('weather_station_eventlayer','weather_station_locations',)
_ = arcpy.analysis.CreateThiessenPolygons(res,'weather_station_polygons',fields_to_copy='ALL')

Merge weather station polygons with the roads. Each road gets assigned a station and the weather feed gets joined to it there.

In [30]:
field_mappings = arcpy.FieldMappings()
field_mappings.addTable('centerlines_merged_with_pop')
features_mapping = arcpy.FieldMap()
features_mapping.addInputField('weather_station_polygons','station_id')
field_mappings.addFieldMap(features_mapping)

_ = arcpy.analysis.SpatialJoin('centerlines_merged_with_pop',
                               'weather_station_polygons',
                               'centerlines_merged_with_weather',
                               join_type='KEEP_ALL',
                               match_option='closest',
                               search_radius='100 miles',
                               field_mapping=field_mappings)


If the event that you'd like to try inverse distance weighting, this is a table of the 5 nearest weather stations to each road segment. 

In [31]:
_ = arcpy.analysis.GenerateNearTable('centerlines_merged',
                                     'weather_station_locations',
                                     'closest_weather_stations',
                                     closest=False,
                                     closest_count=5)
arcpy.management.AlterField(_,'IN_FID','segment_id',clear_field_alias=True)
arcpy.management.AlterField(_,'NEAR_FID','station_id',clear_field_alias=True)
arcpy.management.AlterField(_,'NEAR_DIST','distance',clear_field_alias=True)
arcpy.management.AlterField(_,'NEAR_RANK','rank',clear_field_alias=True)

<Result './utah.gdb\\closest_weather_stations'>

Finally, this is the "static features" dataset that will be used over and over again, joined with realtime feeds to produce accident risk predictions

In [32]:
_ = arcpy.management.CopyFeatures('centerlines_merged_with_weather','static_features')

# Spatially Assign Collisions to Roads
This is part of building the training set. We join our accident records to the road so that we have a set of static features for each record.

In [33]:
field_mappings = arcpy.FieldMappings()
field_mappings.addTable('collisions_projected')

features_mapping = arcpy.FieldMap()
features_mapping.addInputField('static_features','segment_id')
field_mappings.addFieldMap(features_mapping)

_ = arcpy.analysis.SpatialJoin('collisions_projected',
                               'static_features',
                               'collisions_joined',
                               join_type='KEEP_COMMON', # Right join
                               match_option='closest',
                               search_radius='50 feet',
                               field_mapping=field_mappings)